<a href="https://colab.research.google.com/github/high-tech-r/rag/blob/main/Udemy_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install openai==1.25.1

In [19]:
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata
import time  # time モジュールをインポート

api_key = userdata.get('OPENAI_API_KEY')


if api_key:
  os.environ["OPENAI_API_KEY"] = api_key
else:
  raise ValueError("ユーザーデータに OPENAI_API_KEY が見つかりません。正しく保存されていることを確認してください。")


client = OpenAI()


def vectorize_text(text):
  response = client.embeddings.create(
      input = text,
      model = "text-embedding-3-small"
  )
  return response.data[0].embedding


question = "2023年の第1事業部の売上はどのくらい？"

documents = [
    "2023年上期売上200億円、下期売上300億円",
    "2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円",
    "2024年は全社で1000億円の売上を目指す"
]


vectors = [vectorize_text(doc) for doc in documents]

print(vectors)
# API呼び出しの間に遅延を導入して、レート制限を回避
#vectors = []
#for doc in documents:
#  vectors.append(vectorize_text(doc))
#  time.sleep(5)  # 各呼び出しの間、1秒待機します。必要に応じて調整してください。



question_vector = vectorize_text(question)

print(question_vector)

[[0.04461405426263809, -0.0278403852134943, 0.04678399860858917, -0.01337770652025938, 0.023435398936271667, 0.01756569929420948, 0.022024935111403465, 0.07937656342983246, -0.006623754743486643, -0.029880132526159286, 0.00588054908439517, -0.015493402257561684, -0.08931490778923035, -0.029793335124850273, 0.03372093290090561, -0.017912890762090683, -0.025540243834257126, -0.024194879457354546, -0.053337231278419495, -0.018509624525904655, 0.004274790175259113, 0.00028294039657339454, 0.019518649205565453, 0.012086589820683002, -0.029359346255660057, 0.021753691136837006, -0.011251160874962807, -0.03706264868378639, 0.03376433253288269, -0.0008157634292729199, 0.04457065463066101, -0.037952326238155365, 0.003935736604034901, 0.007480883039534092, -0.03938448801636696, -0.006206040736287832, 0.011262010782957077, 0.036541860550642014, -0.024108080193400383, 0.018856815993785858, -0.0031708311289548874, 0.02130885235965252, 0.023283502086997032, -0.0028155026957392693, 0.0277318879961967

In [20]:

max_similarity = 0
most_similar_index = 0

for index, vector in enumerate(vectors):
  similarity = cosine_similarity([question_vector], [vector])[0][0]
  print(documents[index], ":", similarity)
  if similarity > max_similarity:
    max_similarity = similarity
    most_similar_index = index

print(documents[most_similar_index])

2023年上期売上200億円、下期売上300億円 : 0.6031004598770014
2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円 : 0.8517503924012055
2024年は全社で1000億円の売上を目指す : 0.6196779986448152
2023年第1事業部売上300億円、第2事業部売上150億円、第3事業部売上50億円


In [21]:
prompt = f'''以下の質問に以下の情報をベースにして答えて下さい。
[ユーザーの質問]
{question}

[情報]
'''

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=200
)

print(response.choices[0].text)

-2018年の第1事業部の売上は100万円
-売上は年間15%ずつ上昇

2023年の第1事業部の売上は、2018年の売上100万円をベースに、1年ごとに売上が15%ずつ上昇することを考慮すると、以下の計算式で求めることができます。

2023年の売上 = 100万円 × (1 + 15%)^5 
= 100万円 × 1.15 × 1.15 × 1.15 × 1.15 × 1.15
= 100万円 × 1.759
= 175.9万円

よって、2023年の第1事業部の


## 特定のURLの情報にRAGを使ってアクセスしてチャットボットから回答を得る

In [22]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity


#url = "https://toukei-lab.com/achademy/?page_id=1619"
url = "https://chatterboxvr.com/wordpress/?p=86"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

text_nodes = soup.find_all("div")

joined_text = "".join(t.text.replace("\t", "").replace("\n", "") for t in text_nodes)

## 1文で書かない場合 ##
# t_all = []
# for t in text_nodes:
#   t_all.append(t.text.replace("\t", "").replace("\n", ""))

# joined_text = "".join(t_all)


In [23]:
joined_text

'\r内容をスキップ 俺のおもちゃ箱 俺のおもちゃ箱Homeこのブログの紹介ホームページ「開発現場のストレスを減らすアサーティブ会話術: キッチリ上達する7日間講座」のレビュー \r                                 Bookレビュー\r                                                             「開発現場のストレスを減らすアサーティブ会話術: キッチリ上達する7日間講座」のレビュー                                                              投稿者                                                                            kojiro777                                    2024年10月16日 アサーティブ会話術とは何か、というと自分の意見を飲み込むことで我慢をしたり、一方的に自分の主張を押し通したりせずに、相手を尊重しながら自分の気持ちを伝えるコミュニケーションのこと。上司などの人間関係で言いたいことを言えずに飲み込んでしまう、というのはおそらく誰もが経験していることと思うが、関係を壊すことなくうまいことこちらの要求を伝えることができればそれだけでも心理的な負担は減る。また、我慢をし続けてしまうと結局のところ会社を去るなど、関係性を絶つ以外に解決策がなくなってしまうので会社とっても損だし辞める側にとっても損。アサーティブコミュニケーションとは過度な我慢をせずに人間関係を継続するテクニックともいえる。家庭内でも同様に使えるかもしれないので家族に「絶対許せない！」と不満があるという方は読んでみてもいいかもしれない。どのように対応したら良いのか、具体的な例が載っているため実際のイメージが湧きやすく、実践もしやすいと感じた。特に社会人になってすぐの経験が浅い人に読んでもらいたい。むしろ、私自身は学生の頃に知りたかったと思う。人間関係で基礎的なコミュニケーションの技術なので、社会人が長くなると自然にできている人は多いと思われる。以下目次の中からの引用1日目「アサーティブ」って一体どういうこと？～アサーティブ

In [24]:
len(joined_text)

28731

In [25]:
chunk_size = 400
overlap = 50
chunks = []
start = 0

while start + chunk_size <= len(joined_text):
  chunks.append(joined_text[start:start + chunk_size])
  start += (chunk_size - overlap)

if start < len(joined_text):
  chunks.append(joined_text[-chunk_size:])



In [26]:
for i, chunk in enumerate(chunks):
  print(f"Chunk {i+1} : {chunk[:50]}...{chunk[-50:]}")

内容をスキップ 俺のおもちゃ箱 俺のおもちゃ箱Homeこのブログの紹介ホームページ「開発現場のスト...                          kojiro777               
Chunk 2 :                           kojiro777               ...ィブコミュニケーションとは過度な我慢をせずに人間関係を継続するテクニックともいえる。家庭内でも同様に
Chunk 3 : ィブコミュニケーションとは過度な我慢をせずに人間関係を継続するテクニックともいえる。家庭内でも同様に....素直に伝える3.対等な目線で話す4.結果に責任を持つちなみにこれはPMBOKにおけるプロジェクトマ
Chunk 4 : .素直に伝える3.対等な目線で話す4.結果に責任を持つちなみにこれはPMBOKにおけるプロジェクトマ...っともです。でも、何かしっくりこないんです。私の考えも聞いてもらえませんか。」とか「一度や二度なら聞
Chunk 5 : っともです。でも、何かしっくりこないんです。私の考えも聞いてもらえませんか。」とか「一度や二度なら聞...スも減ってくるでしょう。他にも「NO」の伝え方や「ほめる・ほめられる」「交渉」といった、社会人にとっ
                        
                        ...                                         kojiro777
Chunk 8 :                                          kojiro777...すコメント ※ 名前 ※ メール ※ サイト  次回のコメントで使用するためブラウザーに自分の名前、
                             
                             ...Copyright © All rights reserved | Blogus by Themea
Chunk 11 : Copyright © All rights reserved | Blogus by Themea...                 投稿者                           

## 特定のURLの情報を元に作ったチャンクをベースにRAG

In [27]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = OpenAI()


def scrape_article(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  text_nodes = soup.find_all("div")
  joined_text = "".join(t.text.replace("\t", "").replace("\n", "") for t in text_nodes)

  return joined_text


def chunk_text(text, chunk_size, overlap):
  chunks = []
  start = 0
  while start + chunk_size <= len(text):
    chunks.append(text[start:start + chunk_size])
    start += (chunk_size - overlap)
  if start < len(text):
    chunks.append(text[-chunk_size:])

  return chunks


def vectorize_text(text):
  response = client.embeddings.create(
      input = text,
      model = "text-embedding-3-small"
  )
  return response.data[0].embedding


def find_most_similar(question_vector, vectors, documents):
  similarities = []

  for index, vector in enumerate(vectors):
    similarity = cosine_similarity([question_vector], [vector])[0][0]
    similarities.append([similarity, index])

  similarities.sort(reverse=True, key=lambda x: x[0])
  top_documents = [documents[index] for similarity, index in similarities[:2]]

  return top_documents


def ask_question(question, context):
  prompt = f'''以下の質問に以下の情報をベースにして答えて下さい。
  [ユーザーの質問]
  {question}

  [情報]
  {context}
  '''
  print(prompt)
  response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      max_tokens=200
  )

  return response.choices[0].text


url = "https://toukei-lab.com/achademy/?page_id=1619"
chunk_size = 400
overlap = 50

article_text = scrape_article(url)
text_chunks = chunk_text(article_text, chunk_size, overlap)

vectors = [vectorize_text(doc) for doc in text_chunks]

question = "オーダーメイドプランの価格はいくらですか？"
question_vector = vectorize_text(question)

similar_document = find_most_similar(question_vector, vectors, text_chunks)

answer = ask_question(question, similar_document)
print(answer)


以下の質問に以下の情報をベースにして答えて下さい。
  [ユーザーの質問]
  オーダーメイドプランの価格はいくらですか？

  [情報]
  ['てみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラ', '参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみる試してみるオーダーメイドプランは一人一人の目指す姿を実現できるように徹底的に伴走させていただくプランです・スタアカ専用QAフォームでの24時間以内の返信対応（カリキュラムが終了するまでずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）とのマンツーマンメンタリング（月1回×8ヶ月）・前半4ヶ月でスタアカカリキュラムを終わらせ残りの4ヶ月でオーダーメイドの学習・全コンテンツの受講（ずっと）・運営代表（twitter.com/statistics1012）のUdemyコース5種類の贈呈・プレミアムプラン専用のコミュニティに参加オーダーメイ']
  

オーダーメイドプランの価格は398,000円です。



In [28]:
def find_most_similar(question_vector, vectors, documents):
  similarities = []

  for index, vector in enumerate(vectors):
    similarity = cosine_similarity([question_vector], [vector])[0][0]
    similarities.append([similarity, index])
  similarities.sort(reverse=True, key=lambda x: x[0])
  top_documents = [documents[index] for similarity, index in similarities[:2]]
  print(top_documents)

top_documents = find_most_similar(question_vector, vectors, text_chunks)

['てみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラ', '参加オーダーメイドの学習試してみるオーダーメイドプラン398,000 円 全コース受講し放題フォーラム質問し放題＋αのカリキュラム30分の 1 on 1（8回分）専用コミュニティへの参加オーダーメイドの学習試してみる試してみるオーダーメイドプランは一人一人の目指す姿を実現できるように徹底的に伴走させていただくプランです・スタアカ専用QAフォームでの24時間以内の返信対応（カリキュラムが終了するまでずっと）・1回30分の当サービス運営代表（twitter.com/statistics1012）とのマンツーマンメンタリング（月1回×8ヶ月）・前半4ヶ月でスタアカカリキュラムを終わらせ残りの4ヶ月でオーダーメイドの学習・全コンテンツの受講（ずっと）・運営代表（twitter.com/statistics1012）のUdemyコース5種類の贈呈・プレミアムプラン専用のコミュニティに参加オーダーメイ']
